# Hyperliquid API Exploration

This notebook is dedicated to exploring the Hyperliquid API and its functionalities. It allows users to interact with the API and analyze the data it provides. Different to Dune in that its based on RESTAPI.

In [ ]:
import sys
import os
import pandas as pd
from datetime import datetime

# Add project root to Python path
project_root = os.path.abspath('..')  # Go up one level from notebooks/
sys.path.insert(0, project_root)

print(f"Added to Python path: {project_root}")
print(f"Current working directory: {os.getcwd()}")

from src.data_providers.factory import setup_providers
from datetime import datetime

# Setup providers
manager = setup_providers()

# Test connections
print("Connection Status:")
for provider, status in manager.test_all_connections().items():
    print(f"  {provider}: {'✅ Connected' if status else '❌ Failed'}")

# Get Hyperliquid provider from the manager
hl = manager.get_provider('hyperliquid')

if hl:
    print("\n🚀 Testing Hyperliquid API endpoints:")
    
    # 1. Get available symbols
    symbols = hl.get_available_symbols()
    print(f"📊 Available symbols: {symbols[:10]}...")  # Show first 10
    
    # 2. Get current mid prices
    mids = hl.get_all_mids()
    print(f"\n💰 Current mid prices:")
    print(mids.head())
    
    # 3. Get market data for ETH
    eth_data = hl.get_market_data('ETH', interval='1h', lookback_hours=48)
    print(f"\n📈 ETH OHLCV data (last 48h):")
    print(f"Shape: {eth_data.shape}")
    print(eth_data.tail())
    
    # 4. Get funding rates
    funding = hl.get_funding_rates()
    print(f"\n💸 Funding rates:")
    print(funding.head())
    
    # 5. Save data for analysis
    if not eth_data.empty:
        data_dir = os.path.join(project_root, 'data', 'raw', 'hyperliquid')
        os.makedirs(data_dir, exist_ok=True)
        
        timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
        file_path = os.path.join(data_dir, f'eth_ohlcv_{timestamp}.parquet')
        
        eth_data.to_parquet(file_path)
        print(f"\n✅ Data saved: {file_path}")

else:
    print("❌ Hyperliquid provider not available")

## Data Exploration

Once the data is fetched, we can explore its structure and contents.

Getting market data directly through HL API

In [ ]:
# Cell 5 - Using Hyperliquid API directly (for testing/exploration)
import requests
import pandas as pd
from datetime import datetime, timedelta
import json

def get_hyperliquid_market_data_direct(symbol: str, interval: str = '1h', lookback_hours: int = 24):
    """
    Get market data directly from Hyperliquid API
    
    Args:
        symbol: Trading symbol (e.g., 'HYPE', 'ETH')
        interval: Time interval ('1m', '1h', '1d')
        lookback_hours: How many hours back to fetch
    """
    try:
        # Hyperliquid API endpoint for candles
        url = "https://api.hyperliquid.xyz/info"
        
        # Calculate start time
        end_time = int(datetime.now().timestamp() * 1000)
        start_time = int((datetime.now() - timedelta(hours=lookback_hours)).timestamp() * 1000)
        
        # Request payload for candles
        payload = {
            "type": "candleSnapshot",
            "req": {
                "coin": symbol,
                "interval": interval,
                "startTime": start_time,
                "endTime": end_time
            }
        }
        
        print(f"🔍 Fetching {symbol} data directly from Hyperliquid API...")
        print(f"   Interval: {interval}, Lookback: {lookback_hours}h")
        
        # Make the request
        response = requests.post(url, json=payload, timeout=30)
        response.raise_for_status()
        
        data = response.json()
        
        if not data or len(data) == 0:
            print(f"⚠️  No data returned for {symbol}")
            return pd.DataFrame()
        
        # Convert to DataFrame
        df = pd.DataFrame(data)
        
        # Rename columns to standard OHLCV format
        if 't' in df.columns:
            df = df.rename(columns={
                't': 'timestamp',
                'T': 'timestamp_close', 
                'o': 'open',
                'h': 'high', 
                'l': 'low',
                'c': 'close',
                'v': 'volume',
                'n': 'trades'
            })
            
            # Convert timestamp to datetime
            df['datetime'] = pd.to_datetime(df['timestamp'], unit='ms')
            
            # Convert price columns to float
            price_cols = ['open', 'high', 'low', 'close']
            for col in price_cols:
                if col in df.columns:
                    df[col] = pd.to_numeric(df[col], errors='coerce')
            
            # Convert volume to float
            if 'volume' in df.columns:
                df[col] = pd.to_numeric(df['volume'], errors='coerce')
            
            # Sort by timestamp
            df = df.sort_values('timestamp').reset_index(drop=True)
            
        print(f"✅ Retrieved {len(df)} candles for {symbol}")
        return df
        
    except requests.exceptions.RequestException as e:
        print(f"❌ Network error fetching {symbol} data: {e}")
        return pd.DataFrame()
    except Exception as e:
        print(f"❌ Error processing {symbol} data: {e}")
        return pd.DataFrame()

def get_available_symbols_direct():
    """Get available trading symbols directly from Hyperliquid"""
    try:
        url = "https://api.hyperliquid.xyz/info"
        payload = {"type": "meta"}
        
        response = requests.post(url, json=payload, timeout=30)
        response.raise_for_status()
        
        data = response.json()
        
        if 'universe' in data:
            symbols = [asset['name'] for asset in data['universe']]
            return symbols
        else:
            return []
            
    except Exception as e:
        print(f"❌ Error fetching symbols: {e}")
        return []

# Direct API usage
print("🚀 Testing Hyperliquid API directly...")

# Get available symbols
symbols = get_available_symbols_direct()
print(f"📊 Available symbols: {len(symbols)} total")
print(f"First 10: {symbols[:10]}")

# Get market data for HYPE token directly
try:
    hype_data = get_hyperliquid_market_data_direct('HYPE', interval='1h', lookback_hours=48)
    
    if not hype_data.empty:
        print(f"\n📊 HYPE Market Data (Direct API):")
        print(f"Shape: {hype_data.shape}")
        print(f"Columns: {hype_data.columns.tolist()}")
        print(f"Date range: {hype_data['datetime'].min()} to {hype_data['datetime'].max()}")
        print("\nFirst 5 rows:")
        print(hype_data[['datetime', 'open', 'high', 'low', 'close', 'volume']].head())
        print("\nLast 5 rows:")
        print(hype_data[['datetime', 'open', 'high', 'low', 'close', 'volume']].tail())
    else:
        print("❌ No HYPE data available")
        
        # Try with a more common symbol
        print("\n🔄 Trying with ETH instead...")
        eth_data = get_hyperliquid_market_data_direct('ETH', interval='1h', lookback_hours=24)
        
        if not eth_data.empty:
            print(f"✅ ETH data retrieved: {eth_data.shape}")
            print(eth_data[['datetime', 'open', 'high', 'low', 'close', 'volume']].head())
        else:
            print("❌ No ETH data available either")

except Exception as e:
    print(f"❌ Error in direct API call: {e}")

# Show current mid prices directly
try:
    url = "https://api.hyperliquid.xyz/info"
    payload = {"type": "allMids"}
    
    response = requests.post(url, json=payload, timeout=30)
    response.raise_for_status()
    
    mids_data = response.json()
    print(f"\n💰 Current mid prices (first 10):")
    
    # Convert to readable format
    if isinstance(mids_data, dict):
        for i, (symbol, price) in enumerate(mids_data.items()):
            if i < 10:  # Show first 10
                print(f"   {symbol}: ${float(price):,.4f}")
    
except Exception as e:
    print(f"❌ Error fetching mid prices: {e}")

Usual method of retrieving through the factory.py file

In [ ]:
# Standard setup for ALL notebooks
from src.data_providers.factory import setup_providers

# One line setup
hl = manager = setup_providers().get_provider('hyperliquid')


# Get market data for HYPE token
try:
    results = hl.get_market_data('HYPE', interval='1h')
    print(f"\n📊 HYPE Market Data:")
    print(f"Shape: {results.shape}")
    print(results.head())
except Exception as e:
    print(f"❌ Error getting HYPE data: {e}")


In [ ]:
hl.get_market_overview(20)

## Further Analysis

You can add additional cells here to perform further analysis on the fetched data, such as visualizations or statistical summaries.